 # 01c_Build a Gaia list of observable WD target from and generate sky trajectories and plot

- Author Sylvie Dagoret-Campagne
- Creation : December 22th 2024
- update : December 22th 2024
- last update : January 6th 2025 (use conda_py313 on my macbook)

## Ckeck on airmass.org : https://airmass.org/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, quantity_support
plt.style.use(astropy_mpl_style)
quantity_support()
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

from matplotlib.dates import (AutoDateLocator, YearLocator, MonthLocator,
                              DayLocator, WeekdayLocator, HourLocator,
                              MinuteLocator, SecondLocator, MicrosecondLocator,
                              RRuleLocator, rrulewrapper, MONTHLY,
                              MO, TU, WE, TH, FR, SA, SU, DateFormatter,
                              AutoDateFormatter, ConciseDateFormatter)

#%matplotlib inline
import pandas as pd

ModuleNotFoundError: No module named 'astropy.visualization'

In [ ]:
import astropy.units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord
from astropy.coordinates import Angle
from astropy.time import Time, TimezoneInfo,TimeDelta
from astropy.coordinates import Longitude,Latitude

In [ ]:
import datetime as dt
from datetime import datetime
import timezonefinder, pytz
from tzwhere import tzwhere
from calendar import monthrange
import calendar

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
# to view the list of VOTABLE
# Simbad.list_votable_fields()

In [ ]:
Simbad.add_votable_fields('sptype')
Simbad.add_votable_fields('flux(U)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(B)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(V)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(R)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(I)') # add the Vega unit Magnitude 
#Simbad.add_votable_fields('flux_unit(V)')
#Simbad.add_votable_fields('flux_unit(I)')
#Simbad.add_votable_fields('flux_system(V)')
#Simbad.add_votable_fields('flux_system(I)')
#Simbad.add_votable_fields('ubv')  # Johnson UBV system

In [ ]:
plt.rcParams["axes.labelsize"]="large"
plt.rcParams["axes.linewidth"]=2.0
plt.rcParams["xtick.major.size"]=8
plt.rcParams["ytick.major.size"]=8
plt.rcParams["ytick.minor.size"]=5
plt.rcParams["xtick.labelsize"]="large"
plt.rcParams["ytick.labelsize"]="large"

plt.rcParams["figure.figsize"]=(12,8)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
#plt.rcParams['axes.facecolor'] = 'blue'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['lines.markeredgewidth'] = 0.3 # the line width around the marker symbol
plt.rcParams['lines.markersize'] = 5  # markersize, in points
plt.rcParams['grid.alpha'] = 0.75 # transparency, between 0.0 and 1.0
plt.rcParams['grid.linestyle'] = '-' # simple line
plt.rcParams['grid.linewidth'] = 0.4 # in points
plt.rcParams['font.size'] = 13

In [ ]:
def CalculateBounds(theta,thetamin,thetamax):
    """
    Handle the boundaries of angles
    
    parameters:
        theta : the input angle in degree to test
        thetamin: lower bound angle
        thetamax: upper bound angle
    """
    
    theta_min_angle = Longitude(thetamin,unit=u.degree)
    theta_max_angle = Longitude(thetamax,unit=u.degree)
    theta_angle = Longitude(theta,unit=u.degree)
    
    #print("longitude angles",theta_angle,theta_min_angle,theta_max_angle)
    
    wrap = 0
    
    if theta_min_angle.degree < theta_max_angle.degree:
        wrap = 360 * u.deg
    else:
        wrap = 180 * u.deg
        
    theta_min_angle.wrap_angle = wrap
    theta_max_angle.wrap_angle = wrap
    theta_angle.wrap_angle = wrap
       
    #print("wrap",wrap,"theta=",theta_angle,"theta_min=",theta_min_angle,"theta_max",theta_max_angle)
    return theta_angle.is_within_bounds(theta_min_angle,theta_max_angle)
    

## Configuration

### Specify the observation date

In [ ]:
NYEAR=2025
NMONTH=1
NDAY=6

## Initialisation

### Location of sites

In [ ]:
list_of_observation_sites = EarthLocation.get_site_names()

In [ ]:
print(list_of_observation_sites)

#### Greenwitch Observatory

In [ ]:
latitude_greenwitch = 51.476852*u.degree 
# There is a little shift between the prime meridian and the Greenwitch observatory
#longitude_greenwitch = -0.000500*u.degree
longitude_greenwitch = -0.00*u.degree
altitude_greenwitch = 68.0*u.m
site_Greenwitch = EarthLocation(lat=latitude_greenwitch, lon=longitude_greenwitch, height=altitude_greenwitch)

In [ ]:
site_Greenwitch 

In [ ]:
tf = timezonefinder.TimezoneFinder()
timezone_greenwitch_str = tf.certain_timezone_at(lat=latitude_greenwitch/u.degree , lng=longitude_greenwitch/u.degree)
print(f"Time zone at Greenwitch Observatory: {timezone_greenwitch_str}")

In [ ]:
# Not working (want to know where is the observatory relative to zone border)
tz_data = tf.get_geometry(tz_name=timezone_greenwitch_str)[0][0]
all_long,all_lat = np.array(tz_data)
min_longitude_tzgreen = all_long.min()
max_longitude_tzgreen = all_long.max()

In [ ]:
print(f" Check longitude : {min_longitude_tzgreen} < {longitude_greenwitch} < {max_longitude_tzgreen}")

#### Rubin-LSST Observatory

In [ ]:
LSSTNAME='Cerro Pachon'
site_LSST = EarthLocation.of_site(LSSTNAME)
longitude_lsst = site_LSST.lon
latitude_lsst = site_LSST.lat
altitude_lsst = site_LSST.height
site_LSST

In [ ]:
cerro_pachon = EarthLocation(lat = latitude_lsst, lon = longitude_lsst, height = altitude_lsst)
cerro_pachon 

In [ ]:
tf = timezonefinder.TimezoneFinder()
timezone_lsst_str = tf.certain_timezone_at(lat=latitude_lsst/u.degree , lng=longitude_lsst/u.degree)
print(f"Time zone at Rubin LSST Observatory: {timezone_lsst_str}")

In [ ]:
# Not working (want to know where is the observatory relative to zone border)
tz_data = tf.get_geometry(tz_name=timezone_lsst_str)[0][0]
all_long,all_lat = np.array(tz_data)
min_longitude_tzlsst = all_long.min()
max_longitude_tzlsst = all_long.max()

In [ ]:
print(f" Check longitude : {min_longitude_tzlsst} < {longitude_lsst} < {max_longitude_tzlsst}")

### UTC offset

#### UTC offset using longitude

In [ ]:
longitude_offset = longitude_lsst - longitude_greenwitch
longitude_offset

In [ ]:
longitude_offset.hour

In [ ]:
#utcoffset_number = timezone.utcoffset(dt).total_seconds()/60./60.
utcoffset_number1 = longitude_offset.hour
print("The UTC offset in Chile is ",utcoffset_number1," hours")

In [ ]:
utcoffset_number2  = (longitude_lsst * 24.0 / 360.0/u.deg)
utcoffset_number2 

- Notice that this offset calculated above is in sidereal time not in solar time

#### UTC offset Using timezone

In [ ]:
if timezone_lsst_str is None:
    print("Could not determine my time zone")
else:
    # Display the current time in that time zone
        
    timezone_lsst = pytz.timezone(timezone_lsst_str)
    timezone0 = pytz.timezone('UTC')
    
    dt = datetime.utcnow()
    utc_offset =  timezone_lsst.utcoffset(dt)
    
    print("\t - The UTC Time now %s" % dt)
    print(f"\t - the UTC offset in zone {timezone_lsst_str} is {utc_offset}")

In [ ]:
# This is the offset that works with the sidereal time calculated after
utcoffset_number3 = timezone_lsst.utcoffset(dt).total_seconds()/60./60.
utcoffset_number3

### Try to correct internally to the time zone

In [ ]:
dt_corr = (longitude_offset.hour - np.floor(longitude_offset.hour))
dt_corr

In [ ]:
utcoffset_number4 = utcoffset_number3 - dt_corr

#### the TimezoneInfo object from astropy

- https://docs.astropy.org/en/stable/api/astropy.time.TimezoneInfo.html
- create the TimezoneInfo object required for sideral time

Have to choose among the four calculations :**utcoffset_number1, utcoffset_number2, utcoffset_number3, utcoffset_number4**

- Only utcoffset_number3 gives the correct sidereal time within 2 minutes

In [ ]:
utcoffset_number = utcoffset_number3
tz_utc_minus_xx_hours = TimezoneInfo(utc_offset = utcoffset_number*u.hour)

### Night of observation and local time

- creation of an observation date chosen very close to the local midnight from the chosen NYEAR,NMONTH,NDAY
- Note we provide the **tz_utc_minus_xx_hours** which provide the UTC offset of the site

In [ ]:
# datetime at local midnight in the timezone of Chile by providing the right TimezoneInfo() object set at site Timezone
night_obs = datetime(NYEAR, NMONTH, NDAY, 0, 0)
night_obs_midnight = datetime(NYEAR, NMONTH, NDAY, 23, 59,59,tzinfo = tz_utc_minus_xx_hours)
night_obs_midnight

In [ ]:
night_obs_str = night_obs.strftime("%Y-%m-%d %H:%M:%S")
print(f"Night of Observation : {night_obs_str}")

In [ ]:
#check the string of the date
night_obs_midnight_str = night_obs_midnight.strftime("%Y-%m-%d %H:%M:%S")
night_obs_midnight_str
print(f"Midnight of Night of Observation : {night_obs_midnight_str}")

### Compute UTC and Sideral Time at local midnight

We check here that the offset in time between UTC and Chile time is 3 hours

##### TimezoneInfo object from astropy
https://docs.astropy.org/en/stable/api/astropy.time.TimezoneInfo.html

In [ ]:
tz_utc = TimezoneInfo() 
print("\t - Local time in Santiago in (GMT-3) in winter : ",night_obs_midnight)
print("\t - Time in UTC at the Local Time above         : ",night_obs_midnight.astimezone(tz_utc))

#### Sidereal time at LSST site 

- https://docs.astropy.org/en/stable/time/index.html
- we compute here the local sideral time at midnight local time
- we need to provide the utc offset  from the object TimezoneInfo required later to compute the local siteral time

#### Sidereal time and angle at local time midnight

- this calculation has been checked in Stellarium at Cerro Pachon for this observation date and time
- using utc_offset3 based on timezone info, I have only 2 minutes differences

In [ ]:
# the sidereal time is constructed from 1) local clock wall time, 2) the UTC timezone 3) and the location of observation site on earth 
t_lsst = Time(night_obs_midnight.astimezone(tz_utc), scale='utc',location = site_LSST)
#t_sidereal_lsst = t_lsst.sidereal_time('apparent')  
t_sidereal_lsst = t_lsst.sidereal_time('mean')  
print("Sideral Time at LSST midnight:",t_sidereal_lsst)

#### The Sideral angle is measured from the meridian ( westward in north hemisphere)

In [ ]:
print("Sidereal Time angle at LSST midnight:",Angle(t_sidereal_lsst).to(u.degree))
print("Sidereal Time angle at LSST midnight:",Angle(t_sidereal_lsst).degree)

### get Gaia

#### Retrieve all Gaia WD

- list made by Philippe Gris

In [ ]:
df_wd_selected = pd.read_csv("philippecalspecandgaia/star_map_wd_2025_1_10.csv")

In [ ]:
df_wd_selected

In [ ]:
all_mainid = []
for idx,wd_name in enumerate(df_wd_selected.target.values):
    result_table = Simbad.query_object(wd_name).to_pandas()
    main_id = result_table.iloc[0]["MAIN_ID"]
    all_mainid.append( main_id) 

In [ ]:
df_wd_selected["MAIN_ID"] = all_mainid 

In [ ]:
df_wd_selected

### Search WD among Gaia source with spectra

#### Search for all names of targets

In [ ]:
df_wd_selected_names = df_wd_selected["MAIN_ID"].values

In [ ]:
for idx,name in enumerate(df_wd_selected_names):
    target_name = df_wd_selected.iloc[idx]["target"]
    tab = Simbad.query_objectids(name)
    values = tab["ID"].data
    NID = len(values)
    found_gaia = False
    found_gaia_name = None
    for idx2 in range(NID):
        if 'Gaia DR3' in str(values[idx2]):
            print(idx,target_name, ":: \t " , name , '<==>', values[idx2])
            found_gaia_name = values[idx2]
            found_gaia = True
        elif 'Gaia DR2' in str(values[idx2]):
            print(idx,target_name, ":: \t " ,name , '<==>', values[idx2])
            found_gaia_name = values[idx2]
            found_gaia = True      

#### Access to Corentin's spectral list

In [ ]:
from gaiaspec.getGaia import *

In [ ]:
flag_getsgaia = False
try:
    df = get_gaia_sources()
    flag_getsgaia = True
except:
    filename = "philippecalspecandgaia/star_map_wd_2025_1_10.csv"
    df = pd.read_csv(filename,index_col=0)  
finally:
    print(f">>>> getGaia : {flag_getsgaia} <<<<<")
    #print(df.head())   

#### Associate to the WD list the closest Gaia source (with spectrum)

In [ ]:
all_row_gaia_sel = []
for index, row in df_wd_selected.iterrows():
    ra0 = row["ra"]
    dec0 = row["dec"]
    coord0 = SkyCoord(ra0*u.deg,dec0*u.deg)
    df_copy = df.copy()
    df_copy["sep"] = df_copy.apply(lambda row : coord0.separation(SkyCoord(row["ra"]*u.deg,row["dec"]*u.deg)).arcsec,axis=1)
    sep_min = df_copy["sep"].min()
    row_sep_min = df_copy[df_copy["sep"] == sep_min]
    all_row_gaia_sel.append(row_sep_min) 

In [ ]:
df_gaia_sel = pd.concat(all_row_gaia_sel)

In [ ]:
df_gaia_sel

### Conclusion

- NO WD Match a Gaia source

In [ ]:
assert False

## Filter Obervable targets according RA and magnitude and culmination angle
$$
HA = LST - RA
$$

where $HA$ means Hour Angle, $LST$ means Local sidereal time and $RA$ means Right ascension
- $-6H<HA<+6H \longrightarrow  LST-6H <RA< LST+6H $ 
- Select the target which Right-Asccention is +/- 6 hours from the sideral time
- The culmination Hour angle at the meridian must not be less than -6H to + 6H

#### - Criteria on Margin Angle

- **Strange but I cannot have a larger margin : need to understand**

In [ ]:
MarginAngle = Angle(6.0,u.hour)
MarginAngle

$$
RA = LST - HA(margin\; Angle)
$$

In [ ]:
#calculate tdege range of RA target to be visible within the MarginAngle
#ra_min = (t_sidereal_lsst - MarginAngle).degree
#ra_max = (t_sidereal_lsst + MarginAngle).degree
ra_min = Longitude((t_sidereal_lsst - MarginAngle).degree,unit=u.deg)
ra_max = Longitude((t_sidereal_lsst + MarginAngle).degree,unit=u.deg)

In [ ]:
ra_min

In [ ]:
ra_max

In [ ]:
#ra_min_angle = Angle(ra_min,u.degree)
#ra_max_angle = Angle(ra_max,u.degree)
ra_min_angle = Longitude(ra_min,u.degree)
ra_max_angle = Longitude(ra_max,u.degree)
#ra_min_angle.wrap_angle = 180 * u.deg
#ra_max_angle.wrap_angle = 180 * u.deg

In [ ]:
ra_min_angle.degree

In [ ]:
ra_max_angle.degree

#### - Criteria on culmination angle

- The culmination angle is obtained when HA = 0, then RA = LST

In [ ]:
culmin_angle_min = 40

#### - Criteria on magnitude in V

In [ ]:
magLim = 12.

### Algo for the selection of the target wrt RA selection , culmination angle, 

In [ ]:
latitude = latitude_lsst

all_flag_select = []

# numeric collections
all_ra_angles_deg = []
all_dec_angles_deg = []
all_altmax_angle_deg = []
all_zenithmin_angle_deg = []
all_magV = []

# loop on each entries in the df table to calculate angles
for index, row in df.iterrows(): 
    target_name = row["Star_name"]
    hd_name = row["HD_name"]
    
    # compute numeric values for ra-dec
    ra_angle = Longitude(row['RA'],unit = u.hour) # the RA angle is a string in Sexagesimal hours
    #ra_angle.wrap_angle = 180 * u.deg
    
    dec_angle = Angle(row["Decl"],unit = u.deg) # the dec angle is a string in degrees 

    
    # compute the culmination angle from the zenith angle
    # North hemisphere
    if latitude.deg >= 0:
        if dec_angle.deg >= latitude.deg:
            zenith_angle =   dec_angle - latitude
        else:
            zenith_angle = latitude - dec_angle
    # south hemisphere
    else:
        if dec_angle.deg < latitude.deg :
            zenith_angle =   latitude - dec_angle 
        else:
            zenith_angle =   dec_angle - latitude 
        
    culmination_angle = Angle(90.0, unit = u.deg) - zenith_angle

    
    # don't remember why I do this
    if (index !=12) and (index !=13) and (index!= 95):
        magV = float(row["V"])
    else:
        if index == 12:
            magV = 12.47
        elif index==13:
            magV = 13.80
        elif index == 95:
            magV = 17.01
            
             
    all_ra_angles_deg.append(ra_angle.degree)
    all_dec_angles_deg.append(dec_angle.degree)
    all_altmax_angle_deg.append(culmination_angle.degree) 
    all_zenithmin_angle_deg.append(zenith_angle.degree )
    all_magV.append(magV)
  
    # test on ra angle
    #flag1 = ra_angle.is_within_bounds(ra_min_angle,ra_max_angle)
    flag1 = CalculateBounds(ra_angle.degree,ra_min_angle.degree,ra_max_angle.degree)
    
    flag2 = (magV < magLim)
    flag3 = False
    
    # test on culmination angle
    if culmination_angle.degree > culmin_angle_min :
        flag3 = True
        
    
    flag = flag1 & flag2 & flag3
    
    if target_name == "HD185975":
        # keep polar star
        flag = True

    # test on preselected names
    if FLAG_PRESELECTION_TARGET:
        if target_name in preselected_target_names:
          
            if not flag:
                print(f"BE CAREFULL , a rejected target {target_name} is in Pre-Selection list, PLEASE CHECK")
            flag =True
        else:
            if flag:
                print(f"BE CAREFULL , a good target {target_name} is not in Pre-Selection list, PLEASE CHECK")
            flag = False
        
    
    if flag:
        print(f"{index}, {target_name} ({hd_name}), RA = {ra_angle.hour:.2f} , DEC = {dec_angle.degree:.2f} , RA-MIN-MAX = {ra_min_angle.deg:.1f}-{ra_max_angle.deg:.1f},zenith_angle = {zenith_angle.degree:.2f}  culmination angle = {culmination_angle.degree:.2f} , magV = {magV} , flag_select = {flag}")
   
    all_flag_select.append(flag)                

## Add calculated numerical values to the pandas table

In [ ]:
df["ra_deg"] = all_ra_angles_deg
df["dec_deg"] = all_dec_angles_deg 
df["alt_max"] = all_altmax_angle_deg
df["zenith_min"] = all_zenithmin_angle_deg
df["magV"] = all_magV 

### Example for debug

In [ ]:
cut = df["Star_name"] == "HD205905"

In [ ]:
df[cut]

## Apply all selection cuts on RA, culmination and magnitude

In [ ]:
df["select"] = all_flag_select
cut = df["select"] == True

t=df[cut]
NTargets=len(t)

In [ ]:
t

### Must reindex the table

In [ ]:
t = t.reset_index(drop=True)

In [ ]:
t

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NTargets)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NTargets), alpha=1)

## Compute Target location before computing their sky trajectory

In [ ]:
all_target_names = t["Astroquery_Name"]
print("order in selected target location")
print(all_target_names)
selected_target_locations = [ SkyCoord.from_name(target_name) for target_name in all_target_names]

In [ ]:
all_ra_astroquery=[]
all_dec_astroquery=[]
for target_loc in selected_target_locations:
    all_ra_astroquery.append(target_loc.ra)
    all_dec_astroquery.append(target_loc.dec)

## Compute target tag

- the legend in the plot

In [ ]:
all_target_simbadnames = t["Star_name"]
all_Vmag = []
all_types = []
all_target_tagnames = []
for idx, target_name in enumerate(all_target_simbadnames):
    flag_simbad = False
    try:
        result_table = Simbad.query_object(target_name)
        all_Vmag.append(result_table['FLUX_V'][0])
        all_types.append(result_table['SP_TYPE'][0])
        
        row = t.iloc[idx]
        tagname =  target_name + ", m = " + str(result_table['FLUX_V'][0]) + ", B-V = "+ row["B_V"]  + " (" + result_table['SP_TYPE'][0] +")" 
        flag_simbad = True
    except:
        row = t.iloc[idx]
        tagname = row["Name"] + ", B-V="+ row["B_V"]  + ", m= " + row["V"]
    finally:
        
        print(f"{idx}) {target_name} ::{tagname} :: simbad : {flag_simbad}")
        all_target_tagnames.append(tagname)

In [ ]:
result_table

In [ ]:
all_target_tagnames

In [ ]:
t["tag"] = all_target_tagnames

# Astronomical in Observation frame for each selected target

## Initialisation 

In [ ]:
utcoffset_number

In [ ]:
night_obs_midnight_str

In [ ]:
# Greenwitch time at local midnight time
midnight_utc = Time(night_obs_midnight_str) - utcoffset_number*u.hour
delta_midnight = np.linspace(-12, 12, 1000)*u.hour
sideral_times = Angle(t_sidereal_lsst) + Angle(delta_midnight)

# utc times around local midnight
times_evening_to_morning = midnight_utc + delta_midnight
frame_evening_to_morning = AltAz(obstime=times_evening_to_morning, location=cerro_pachon)

In [ ]:
times_evening_to_morning_datetime = times_evening_to_morning.to_datetime()

## Sun frame

In [ ]:
from astropy.coordinates import get_sun
sunaltazs_evening_to_morning = get_sun(times_evening_to_morning).transform_to(frame_evening_to_morning)

## Moon frame

In [ ]:
# Old version in conda_py310
#from astropy.coordinates import get_moon
#moon_evening_to_morning = get_moon(times_evening_to_morning)
#moonaltazs_evening_to_morning = moon_evening_to_morning.transform_to(frame_evening_to_morning)

In [ ]:
from astropy import coordinates

### Get moon ephemerides
- Checked sky trajectory at https://airmass.org/

In [ ]:
moon_evening_to_morning = coordinates.get_body("moon",times_evening_to_morning,location=site_LSST)
moonaltazs_evening_to_morning = moon_evening_to_morning.transform_to(frame_evening_to_morning)

## Targets Frame

In [ ]:
all_target_altazs_evening_to_morning = [target_location.transform_to(frame_evening_to_morning) for target_location in  selected_target_locations ]

## Distance to the moon

In [ ]:
all_target_distancetomoon_evening_to_morning = [] 

num_target=0
for idx,targetcoordinateseries in enumerate(all_target_altazs_evening_to_morning): 
    N_coord = len(targetcoordinateseries)
    idx_coord = 0
    all_distances_sep = []
    for idx_coord in range(N_coord):
        distance_sep=targetcoordinateseries[idx_coord].separation(moonaltazs_evening_to_morning[idx_coord])
        all_distances_sep.append(distance_sep.degree)
    all_distances_sep= np.array(all_distances_sep)
    all_distances_sep_min = all_distances_sep.min()
    all_target_distancetomoon_evening_to_morning.append(all_distances_sep_min) 
    all_target_tagnames[idx] += f", d_m = {all_distances_sep_min:.0f}°"
    num_target+=1

In [ ]:
all_target_tagnames

### Update target tag

In [ ]:
t["tag"] = all_target_tagnames

In [ ]:
t

## Sort target according increasing max culminating time

In [ ]:
all_timemax=np.zeros(NTargets)
all_altitudesmax=np.zeros(NTargets)
for idx in np.arange(NTargets):
    altitudes=all_target_altazs_evening_to_morning[idx].alt
    idx_max=np.where(altitudes==altitudes.max())[0][0]
    all_timemax[idx]=delta_midnight[idx_max].value
    all_altitudesmax[idx]=altitudes.max().degree

In [ ]:
all_altitudesmax

In [ ]:
t["tmax"] = all_timemax

In [ ]:
all_timemax

In [ ]:
rank = np.argsort(all_timemax)
rank

In [ ]:
rankinv = np.argsort(rank)
rankinv

In [ ]:
#t.index[rank]

In [ ]:
t["rank_tmax"] = rankinv # rank in time
t["rank_num"] = np.arange(NTargets) # rank in original table

In [ ]:
#t = t.sort_values(by = ['tmax'])

### Must reindex dataframe for loop on targets (in sorting order)

In [ ]:
#t.reset_index(inplace=True)

In [ ]:
t

# Plot

In [ ]:
all_target_names = t["Star_name"].values
all_target_names

In [ ]:
fig = plt.figure(figsize=(18,12),layout="constrained")


# First Axis : target sky trajectories
# ====================================
ax=fig.add_subplot(1,1,1)

# plot sun
ax.plot(delta_midnight, sunaltazs_evening_to_morning.alt, color='r', ls=":",label='Sun',lw=5)
#plot moon
ax.plot(delta_midnight, moonaltazs_evening_to_morning.alt, color=[0.75]*3, ls='--', label='Moon',lw=5)


# loop on selected targets in pandas dataframe t according rank order
for index in range(NTargets):
    
    # select the good row in pandas dataframe from time ordering
    row = (t[t["rank_tmax"] == index]).iloc[0]
    

    # get its tag
    label = str(index+1) + ") : " + row["tag"]
    
    # rank in all_target_altazs_evening_to_morning table
    rank_num = row["rank_num"]
    tmax = row["tmax"]
    altmax = row["alt_max"]
    
    
  
    # plot trajectory
    ax.plot(delta_midnight, all_target_altazs_evening_to_morning[rank_num].alt,label=label, lw=2.5,color=all_colors[index])

    # plot label
    if all_target_altazs_evening_to_morning[rank_num].alt[-1]>0:
        ax.text(delta_midnight[-1], all_target_altazs_evening_to_morning[rank_num].alt[-1], f'{index+1}',color=all_colors[index],fontsize=20)
    if all_target_altazs_evening_to_morning[rank_num].alt[0]>0:
        ax.text(delta_midnight[0], all_target_altazs_evening_to_morning[rank_num].alt[0], f'{index+1}',color=all_colors[index],fontsize=20)
    if altmax>0 :
        ax.text(tmax, altmax, f'{index+1}',color=all_colors[index],fontsize=20)
    


ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,sunaltazs_evening_to_morning.alt < -0*u.deg, color='0.5', zorder=0)
ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,sunaltazs_evening_to_morning.alt < -18*u.deg, color='k', zorder=0)

ax.legend(loc='upper right')
ax.set_xlim(-12*u.hour, 12*u.hour)
ax.set_xticks((np.arange(13)*2-12)*u.hour)
ax.set_ylim(0*u.deg, 90*u.deg)
ax.set_xlabel('Hours from Midnight local time')
ax.set_ylabel('Altitude [deg]')
ax.grid(None)
tick_spacing = 1
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
title = "Observations at AuxTel at night " + night_obs_str.split(" ")[0]
ax.set_title(title)
#for label in ax.get_xticklabels(which='major'):
#    label.set(rotation=30, horizontalalignment='right')


# second horizontal axis : Sideral time axis
# ==========================================
ax2 = ax.twiny()
ax2.plot([sideral_times[0].hour,sideral_times[-1].hour],[0,0])
ax2.set_xlabel("sideral time (hour)")
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
ax2.grid(None)

# third horizontal axis : UTC time axis
# =====================================

ax3 = ax.twiny()
# Move twinned axis ticks and label from top to bottom
ax3.xaxis.set_ticks_position("bottom")
ax3.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
offset = -0.12
ax3.spines["bottom"].set_position(("axes", offset))

ax3.set_frame_on(True)
ax3.patch.set_visible(False) # mandatory
for sp in ax3.spines.values():
    sp.set_visible(False)
ax3.spines["bottom"].set_visible(True)
#fmt = '%m/%d %H:%M'
fmt = '%H:%M'
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter(fmt))
ax3.xaxis.set_major_locator(mdates.HourLocator(interval=1))

curvetoremove, = ax3.plot_date(times_evening_to_morning_datetime, sunaltazs_evening_to_morning.alt, color='r', ls=":",lw=0)
curvetoremove.remove()
ax3.set_xlim(times_evening_to_morning_datetime[0], times_evening_to_morning_datetime[-1])



ax3.tick_params(which='major', width=1.00, length=5)
ax3.tick_params(which='minor', width=0.75, length=2.5)
ax3.grid(True)
#plt.gcf().autofmt_xdate()  # orient date labels at a slant
# slant for this axis
for label in ax3.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

ax3.set_xlabel("UTC Time")

if FLAG_PRESELECTION_TARGET: 
    figname="AuxtelStarAlt_preselectedcalspec{:4d}_{:d}_{:d}.png".format(NYEAR,NMONTH,NDAY)
else:
    figname="AuxtelStarAlt_visiblecalspec{:4d}_{:d}_{:d}.png".format(NYEAR,NMONTH,NDAY)

plt.tight_layout()
plt.savefig(figname)

plt.show()

# To plot in staralt

https://www.ing.iac.es//Astronomy/telescopes/wht/catformat.html

In [ ]:
t_staralt = t[["Star_name","RA","Decl","rank_tmax"]]
t_staralt["equinox"] = np.full(NTargets,"J2000")
t_staralt = t_staralt.sort_values(by = ['rank_tmax'])
t_staralt = t_staralt.drop(['rank_tmax'], axis=1)

In [ ]:
t_staralt

In [ ]:
t_staralt.to_csv("staralt.csv",index=False,header=False, sep = " ")

In [ ]:
! cat staralt.csv

# Spheric plot

In [ ]:
def makePolarPlot(azimuthsInDegrees, zenithAngles, marker=".",ax=None,
                       title=None, color=None, objName=None,lw=0.5):
    if ax==None:
        _ = plt.figure(figsize=(20, 10))
        ax = plt.subplot(111, polar=True)
        
    ax.plot([a*np.pi/180 for a in azimuthsInDegrees], zenithAngles, marker, c=color, label=objName,lw=lw)
    if title:
        ax.set_title(title, va='bottom')
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlim(0, 90)
    return ax

In [ ]:
_ = plt.figure(figsize=(20, 10))

ax = plt.subplot(111, polar=True)


darkTimeIndex=[sunaltazs_evening_to_morning.alt < -18*u.deg][0]

# compute zenith and azimuth angles
moon_azimuthsInDegrees = Angle(moonaltazs_evening_to_morning.az).deg
moon_zenithAngles = Angle(moonaltazs_evening_to_morning.zen).deg

#ax=makePolarPlot(moon_azimuthsInDegrees, moon_zenithAngles,ax=ax, marker="--",color="y", title = None, lw=1)

moon_azimuthsInDegreesNight=moon_azimuthsInDegrees[darkTimeIndex]
moon_zenithAnglesNight=moon_zenithAngles[darkTimeIndex]
ax=makePolarPlot(moon_azimuthsInDegreesNight, moon_zenithAnglesNight,ax=ax ,marker="*-",title=None, color = "y",objName="Moon",lw=10)

    
# loop on selected targets in pandas dataframe t according rank order
for index in range(NTargets):
    
    # select the good row in pandas dataframe from time ordering
    row = (t[t["rank_tmax"] == index]).iloc[0]
    

    # get its tag
    label = str(index+1) + ") : " + row["tag"]
    
    # rank in all_target_altazs_evening_to_morning table
    rank_num = row["rank_num"]
    tmax = row["tmax"]
    altmax = row["alt_max"]
    
    
    
    
    
    # compute zenith and azimuth angles
    azimuthsInDegrees=Angle(all_target_altazs_evening_to_morning[rank_num].az).deg
    zenithAngles = Angle(all_target_altazs_evening_to_morning[rank_num].zen).deg
    
    ax=makePolarPlot(azimuthsInDegrees, zenithAngles,ax=ax, marker="-",title=None, color=all_colors[index], lw=0.5)
    
    azimuthsInDegreesNight=azimuthsInDegrees[darkTimeIndex]
    zenithAnglesNight=zenithAngles[darkTimeIndex]
    ax=makePolarPlot(azimuthsInDegreesNight, zenithAnglesNight,ax=ax ,marker="*-",title=None, color=all_colors[index],objName=label,lw=0.5)
    
    thex=zenithAnglesNight[0]*np.sin( (azimuthsInDegreesNight[0]) *np.pi/180.)
    they=zenithAnglesNight[0]*np.cos( (azimuthsInDegreesNight[0])*np.pi/180.)
   
    #plt.text(thex, they,f'{rank}',color=all_colors[rank],fontsize=20)
    
 
ax.legend(bbox_to_anchor=(1.08, 1), prop={'size': 15}, loc='upper left')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_rlim(0, 90)
ax.set_xticklabels(['N', '', 'E', '', 'S', '', 'W', ''])
title = "Observations at AuxTel at night " + night_obs_str.split(" ")[0]
ax.set_title(title)

# Spectra

In [ ]:
from getCalspec.getCalspec import *

In [ ]:
plt.rcParams["figure.figsize"]=(8,4)

In [ ]:
NTargets

In [ ]:
ncols = 2

if NTargets % ncols == 0:
    nrows = NTargets//ncols+1
else:
    nrows = int(np.floor(NTargets/ncols))+1

In [ ]:
nrows

In [ ]:
XMIN = 300.
XMAX = 1100.

In [ ]:
fig, axes= plt.subplots(nrows=nrows,ncols=ncols,sharex=False,figsize=(16,20), layout='constrained')

# loop on selected targets in pandas dataframe t according rank order
all_wl = []
all_fl = []
all_obj= []

for index, ax in enumerate(axes.flat):
    
    if index < NTargets:
    
        # select the good row in pandas dataframe from time ordering
        row = (t[t["rank_tmax"] == index]).iloc[0]
    

        # get its tag
        label = str(index+1) + ") : " + row["tag"]
    
        # rank in all_target_altazs_evening_to_morning table
        target_name = row["Star_name"]
    
        test = is_calspec(target_name)
    
        if test:
        
            c = Calspec(target_name)
            c.get_spectrum_fits_filename()  # download the fits file
            tab = c.get_spectrum_table()  # download and return an Astropy table
            arr = c.get_spectrum_numpy()  # download and return a dictionnary of numpy arrays with units
            #c.plot_spectrum()  # download and plot the spectrum
            
            wl = arr['WAVELENGTH'].to_value()/10.
            fl = arr['FLUX'].to_value()*10.
            
            indexes = np.where(np.logical_and(wl>=XMIN,wl<=XMAX))[0]
            
            wl = wl[indexes]
            fl = fl[indexes]
            ax.plot(wl,fl,"-",color=all_colors[index],label=label)
            ax.set_yscale("log")
            title = f"{index+1}) : {target_name}"
            ax.set_title(title)
            ax.legend()
            ax.set_xlim(XMIN,XMAX)
            
            all_wl.append(wl)
            all_fl.append(fl)
            all_obj.append(title)
        else:
            print(f"{target_name} NOT A CALSPEC")
    else:
        if index == NTargets:
            for j in range(NTargets):
                ax.plot(all_wl[j], all_fl[j],'-',color=all_colors[j],label=all_obj[j] )
            ax.set_yscale("log")
            ax.legend()
            ax.set_xlim(XMIN,XMAX)
#plt.tight_layout()            

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,10))

for j in range(NTargets):
    ax.plot(all_wl[j], all_fl[j],'-',color=all_colors[j],label=all_obj[j] )
    ax.set_yscale("log")
    ax.legend()
    ax.set_xlim(XMIN,XMAX)
title = "Possible target spectra at AuxTel for night " + night_obs_str.split(" ")[0]
ax.set_title(title) 
ax.set_xlabel("$\\lambda$ (nm)")
ax.set_ylabel("FLAM")

## Choose spectra

- **Select by hands the target one want to select**

In [ ]:
index_chosen_spectra = np.array([10,12,15]) -1

In [ ]:
fig = plt.figure(figsize=(18,12),layout="constrained")
ax=fig.add_subplot(1,1,1)


# First Axis : target sky trajectories
# ====================================

# plot sun
ax.plot(delta_midnight, sunaltazs_evening_to_morning.alt, color='r', ls=":",label='Sun',lw=5)
#plot moon
ax.plot(delta_midnight, moonaltazs_evening_to_morning.alt, color=[0.75]*3, ls='--', label='Moon',lw=5)


# loop on selected targets in pandas dataframe t according rank order
for index in range(NTargets):
    
    if index not in index_chosen_spectra:
        continue
    
    # select the good row in pandas dataframe from time ordering
    row = (t[t["rank_tmax"] == index]).iloc[0]
    

    # get its tag
    label = str(index+1) + ") : " + row["tag"]
    
    # rank in all_target_altazs_evening_to_morning table
    rank_num = row["rank_num"]
    tmax = row["tmax"]
    altmax = row["alt_max"]
    
    
  
    # plot trajectory
    ax.plot(delta_midnight, all_target_altazs_evening_to_morning[rank_num].alt,label=label, lw=2.5,color=all_colors[index])

    # plot label
    if all_target_altazs_evening_to_morning[rank_num].alt[-1]>0:
        ax.text(delta_midnight[-1], all_target_altazs_evening_to_morning[rank_num].alt[-1], f'{index+1}',color=all_colors[index],fontsize=20)
    if all_target_altazs_evening_to_morning[rank_num].alt[0]>0:
        ax.text(delta_midnight[0], all_target_altazs_evening_to_morning[rank_num].alt[0], f'{index+1}',color=all_colors[index],fontsize=20)
    if altmax>0 :
        ax.text(tmax, altmax, f'{index+1}',color=all_colors[index],fontsize=20)
    


ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,sunaltazs_evening_to_morning.alt < -0*u.deg, color='0.5', zorder=0)
ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,sunaltazs_evening_to_morning.alt < -18*u.deg, color='k', zorder=0)

ax.legend(loc='upper right')
ax.set_xlim(-12*u.hour, 12*u.hour)
ax.set_xticks((np.arange(13)*2-12)*u.hour)
ax.set_ylim(0*u.deg, 90*u.deg)
ax.set_xlabel('Hours from Midnight local time')
ax.set_ylabel('Altitude [deg]')
ax.grid(None)
tick_spacing = 1
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
title = "Observations at AuxTel at night " + night_obs_str.split(" ")[0]
ax.set_title(title)


#Second axis : Sideral time axis
# ==============================

ax2 = ax.twiny()
ax2.plot([sideral_times[0].hour,sideral_times[-1].hour],[0,0])
ax2.set_xlabel("sideral time (hour)")
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
ax2.grid(None)


# Third horizontal axis : UTC time
# ================================

ax3 = ax.twiny()
# Move twinned axis ticks and label from top to bottom
ax3.xaxis.set_ticks_position("bottom")
ax3.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
offset = -0.12
ax3.spines["bottom"].set_position(("axes", offset))

ax3.set_frame_on(True)
ax3.patch.set_visible(False) # mandatory
for sp in ax3.spines.values():
    sp.set_visible(False)
ax3.spines["bottom"].set_visible(True)
#fmt = '%m/%d %H:%M'
fmt = '%H:%M'
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter(fmt))
ax3.xaxis.set_major_locator(mdates.HourLocator(interval=1))

curvetoremove, = ax3.plot_date(times_evening_to_morning_datetime, sunaltazs_evening_to_morning.alt, color='r', ls=":",lw=0)
curvetoremove.remove()
ax3.set_xlim(times_evening_to_morning_datetime[0], times_evening_to_morning_datetime[-1])

ax3.tick_params(which='major', width=1.50, length=5)
ax3.tick_params(which='minor', width=0.75, length=2.5)
ax3.grid(True)
# rotation of labels
for label in ax3.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
ax3.set_xlabel("UTC Time")

plt.tight_layout()

figname="AuxtelStarAlt_CalspecSpectraSelected{:4d}_{:d}_{:d}.png".format(NYEAR,NMONTH,NDAY)

plt.savefig(figname)
plt.show()